## Midterm
### Due Friday, October 27 by midnight

This midterm uses the 'nyc_health.db'.

Questions are worth 10 points each. You'll get an additional 10 points just for making the submission!

Tips:
- Work slow; tackle the queries one small piece at a time
- Remember to include the code to create any views you use
- Make sure your tables and their column names are the same as we used in class (i.e., your CREATE TABLE statements should be identical to what we wrote in class). You can name calculated fields whatever you want.
- Double and triple check your .data302 file before submitting

In [32]:
import db
import pandas as pd
import sqlite3

# You'll find this library on blackboard
import dbga

#python libaries specific to making your submission file
from collections import defaultdict as dd
import json

In [33]:
dbga.__version__

'1.0.3'

In [34]:
my_sql = dd(dict)

In [35]:
my_sql['setup'] = []

In [36]:
path_to_db = '../databases/nyc_health.db'

In [37]:
conn = sqlite3.connect(path_to_db)
curs = conn.cursor()

In [38]:
curs.execute("PRAGMA foreign_keys = ON;")
pd.read_sql("PRAGMA foreign_keys;", conn)

,foreign_keys
0,1


---
1) Get a list of all restaurants (by camis) who have been cited for a violation containing the words 'vermin', 'mice', or 'rats'.

Have your query return the following columns:
- camis
- dba
- viol_id
- viol_desc
- A count of the number of times they have been cited for each violation

Order the results with the most violations on top.

In [8]:
sql1 = '''
SELECT "camis","dba","viol_id","viol_desc", count(*) as "A count of the number of times they have been cited for each violation" 
FROM tRest
JOIN tInsp USING(camis)
JOIN tViol USING(viol_id)
WHERE "viol_desc" LIKE '%vermin%'
    OR "viol_desc" LIKE '%mice%'
    OR "viol_desc" LIKE '%rats%'
GROUP BY "camis","dba","viol_id","viol_desc"
ORDER BY "A count of the number of times they have been cited for each violation" DESC
;'''

pd.read_sql(sql1, conn)

,camis,dba,viol_id,viol_desc,A count of the number of times they have been cited for each violation
0,40861669,NEW VICTORY RESTAURANT,08A,Facility not vermin proof. Harborage or condit...,15
1,41430594,1 STOP PATTY SHOP,04L,Evidence of mice or live mice present in facil...,14
2,50033122,PARTY WELL REST & ORIENTAL BAKERY,08A,Facility not vermin proof. Harborage or condit...,14
3,40861669,NEW VICTORY RESTAURANT,04L,Evidence of mice or live mice present in facil...,13
4,41309387,NEW TACO EXPRESS,04L,Evidence of mice or live mice present in facil...,13
...,...,...,...,...,...
29033,50068060,TASTE OF PUNJAB,08A,Facility not vermin proof. Harborage or condit...,1
29034,50068098,SUBWAY,08A,Facility not vermin proof. Harborage or condit...,1
29035,50068305,SONRISA AZUL,08A,Facility not vermin proof. Harborage or condit...,1
29036,50068537,JOE & THE JUICE COLOMBUS,04L,Evidence of mice or live mice present in facil...,1


In [9]:
my_sql['q1']['sql'] = sql1

---
2) For each different cuisine type, determine the most commonly cited violation **excluding viol_id 10F** (this is by far the most common violation, and not very "gross").

Have your query return:

- cuisine
- viol_id
- viol_desc
- Number of violations for their most common violation (except 10F)

Limit those results to cuisine types where the max number of violations is greater than 1000.

Order the results in ascending order by the cuisine type.

In [39]:
sql2 = '''
WITH not_10F AS 
(
    SELECT cuisine, viol_id, COUNT(*) as "Number of violations for their most common violation (except 10F)"
    FROM tRest
    JOIN tInsp USING(camis)
    WHERE viol_id <> '10F'
    GROUP BY cuisine, viol_id
),
rank_viol AS 
(
    SELECT cuisine, viol_id, viol_desc, "Number of violations for their most common violation (except 10F)",
        ROW_NUMBER() OVER (PARTITION BY cuisine ORDER BY "Number of violations for their most common violation (except 10F)" DESC) as rn
    FROM not_10F
    JOIN tViol USING(viol_id)
)
SELECT cuisine, viol_id, viol_desc, "Number of violations for their most common violation (except 10F)"
FROM rank_viol
WHERE rn = 1
GROUP BY cuisine
HAVING "Number of violations for their most common violation (except 10F)" > 1000
ORDER BY cuisine ASC

;'''
pd.read_sql(sql2, conn)

,camis,cuisine,viol_id,viol_desc,Number of violations for their most common violation (except 10F)


In [11]:
my_sql['q2']['sql'] = sql2

---

3) When the action description contains the word 'closed', what are the top 5 associated violations?


Have your query return:

- action_desc
- viol_desc
- total number of violations

Order the results with the largest number of violations on top

In [12]:
sql3 = '''
SELECT "action_desc","viol_desc", COUNT(*) AS "total number of violations"
FROM tRest
    JOIN tInsp USING(camis)
    JOIN tAction USING(action_id)
    JOIN tViol USING(viol_id)
WHERE LOWER(action_desc) LIKE '%closed%'
GROUP BY action_desc, viol_desc
ORDER BY "total number of violations" DESC
LIMIT 5
;'''
pd.read_sql(sql3, conn)


,action_desc,viol_desc,total number of violations
0,Establishment Closed by DOHMH. Violations wer...,Facility not vermin proof. Harborage or condit...,1419
1,Establishment Closed by DOHMH. Violations wer...,Evidence of mice or live mice present in facil...,973
2,Establishment Closed by DOHMH. Violations wer...,Cold food item held above 41Âº F (smoked fish ...,679
3,Establishment Closed by DOHMH. Violations wer...,Live roaches present in facility's food and/or...,615
4,Establishment Closed by DOHMH. Violations wer...,Non-food contact surface improperly constructe...,562


In [13]:
my_sql['q3']['sql'] = sql3 

---

4) Determine which restaurant has been closed the most (action description contains 'closed').

For that camis, have your query return:

- camis
- dba
- viol_desc
- total number of violations per violation type

Include all violations for that camis (not just the ones they were closed for)

Order the results with the most common violation on top

In [14]:
sql4 = '''
WITH CamiMostViol As 
(
    SELECT camis, action_desc, Count(action_desc) as NumViol
    FROM tRest
    JOIN tInsp USING(camis)
    JOIN tAction USING(action_id)
    WHERE action_desc LIKE '%closed%'
    GROUP BY camis
    ORDER BY NumViol DESC 
    LIMIT 1
)

SELECT c.camis, dba, viol_desc, COUNT(*) as NumViol
FROM CamiMostViol c
JOIN tRest ON c.camis = tRest.camis 
JOIN tInsp ON c.camis = tInsp.camis
JOIN tViol ON tInsp.viol_id = tViol.viol_id
GROUP BY c.camis, dba,viol_desc
ORDER BY NumViol DESC
;'''

pd.read_sql(sql4, conn)

,camis,dba,viol_desc,NumViol
0,40922818,EL TAPATIO MEXICAN RESTAURANT,Non-food contact surface improperly constructe...,8
1,40922818,EL TAPATIO MEXICAN RESTAURANT,Facility not vermin proof. Harborage or condit...,7
2,40922818,EL TAPATIO MEXICAN RESTAURANT,Live roaches present in facility's food and/or...,7
3,40922818,EL TAPATIO MEXICAN RESTAURANT,Evidence of rats or live rats present in facil...,5
4,40922818,EL TAPATIO MEXICAN RESTAURANT,Plumbing not properly installed or maintained;...,5
5,40922818,EL TAPATIO MEXICAN RESTAURANT,Cold food item held above 41Âº F (smoked fish ...,4
6,40922818,EL TAPATIO MEXICAN RESTAURANT,Failure to comply with an Order of the Board o...,3
7,40922818,EL TAPATIO MEXICAN RESTAURANT,Food Protection Certificate not held by superv...,3
8,40922818,EL TAPATIO MEXICAN RESTAURANT,Hot food item not held at or above 140Âº F.,3
9,40922818,EL TAPATIO MEXICAN RESTAURANT,The original nutritional fact labels and/or in...,3


In [15]:
my_sql['q4']['sql'] = sql4

COUNT(*) AS "Number of violations",---

5) Group all restaurants by their DBA, and computer the average number of violations per location.

For example, if there were 100 restaurants with the dba "STARBUCKS", and they had a total of 123 violations, then the average number of violations per location would be 123/100=1.23.

Have your query return:
- dba
- Number of locations
- Number of violations
- Avg violations per location

Order the results with the largest number of locations on top

_Watch out for integer division!_

In [16]:
sql5 = """
SELECT dba, COUNT(camis) AS NumLoc, SUM(viol_count) AS NumViol,
  SUM(viol_count) / CAST(COUNT(camis) AS float) AS "Avg violations per location"
FROM (
  SELECT dba, camis, COUNT(*) AS viol_count
  FROM tRest
  JOIN tInsp USING (camis)
  GROUP BY dba, camis
) 
GROUP BY dba
ORDER BY NumLoc DESC
;"""
pd.read_sql(sql5,conn)

,dba,NumLoc,NumViol,Avg violations per location
0,DUNKIN' DONUTS,410,4247,10.358537
1,SUBWAY,352,3729,10.593750
2,STARBUCKS,281,2130,7.580071
3,MCDONALD'S,206,2778,13.485437
4,"DUNKIN' DONUTS, BASKIN ROBBINS",110,1462,13.290909
...,...,...,...,...
20242,$1 PIZZA,1,8,8.000000
20243,#1 SABOR LATINO RESTAURANT,1,55,55.000000
20244,#1 Natural Juice Bar,1,6,6.000000
20245,#1 GARDEN CHINESE,1,27,27.000000


In [17]:
my_sql['q5']['sql'] = sql5

---

6) For restaurants (by DBA) with only a single location (i.e., DBA with exactly 1 associated CAMIS) and more than 10 total violations, what is their top violation, and how many times have they been cited for it?

Note that a restaurant might have been cited 1 time each for 10 different types of violations, and 2 times for another violation. That would be 12 total violations, which counts as more than 10.

**Limit your results** to cases where their top violation occurred greater than or equal to 15 times.

Have your query return:

- camis
- dba
- viol_desc
- number of violations

Order your results by number of violations, descending.

In [18]:
sql6 = '''
WITH SingleRest AS 
(
    SELECT camis, dba, viol_desc, COUNT(*) as NumViol,
        ROW_NUMBER() OVER (PARTITION BY camis ORDER BY COUNT(*) DESC) as rn
    FROM tRest
    JOIN tINSP USING(camis)
    JOIN tViol USING(viol_id)
    GROUP BY camis, dba, viol_desc
    HAVING COUNT(DISTINCT camis) = 1 
) 

SELECT camis, dba, viol_desc, NumViol
FROM SingleRest
WHERE camis in 
(
    SELECT camis
    FROM SingleRest
    GROUP BY camis
    HAVING SUM(NumViol) > 10
)
    AND rn = 1 AND NumViol >= 15
ORDER BY NumViol DESC
;'''

pd.read_sql(sql6, conn)


,camis,dba,viol_desc,NumViol
0,40861669,NEW VICTORY RESTAURANT,Facility not vermin proof. Harborage or condit...,15


In [19]:
my_sql['q6']['sql'] = sql6

---

7) Considering all possible combinations of violation/action, how many of those combinations **do not occur** in the data?

Have your query return just the total number.

In [20]:
sql7 = '''
WITH ActCombos AS 
(
    SELECT * 
    FROM tViol
    CROSS JOIN tAction
),
ViolCombos As 
(
    SELECT DISTINCT viol_id, action_id
    FROM tInsp 
)
SELECT COUNT(*) AS Missing
FROM ActCombos 
LEFT JOIN ViolCombos
On ActCombos.viol_id = ViolCombos.viol_id 
    AND ActCombos.action_id = ViolCombos.action_id
WHERE ViolCombos.viol_id IS NULL AND ViolCombos.action_id IS NULL 
;'''

pd.read_sql(sql7, conn)

,Missing
0,221


In [21]:
my_sql['q7']['sql'] = sql7

---

8) Return the action and violation descriptions associated with the previous question.

Have your query return:

- action_desc
- viol_desc

Order your results in ascending order by viol_desc

In [22]:
sql8 = '''
WITH ActCombos AS 
(
    SELECT *
    FROM tViol 
    CROSS JOIN tAction 
),
ViolCombos AS 
(
    SELECT DISTINCT viol_id, action_id
    FROM tInsp
)
SELECT a.action_desc, v.viol_desc
FROM ActCombos
LEFT JOIN ViolCombos ON ActCombos.viol_id = ViolCombos.viol_id AND ActCombos.action_id = ViolCombos.action_id
LEFT JOIN tAction a ON ActCombos.action_id = a.action_id
LEFT JOIN tViol v ON ActCombos.viol_id = v.viol_id
WHERE ViolCombos.viol_id IS NULL AND ViolCombos.action_id IS NULL
GROUP BY a.action_desc, v.viol_desc
ORDER BY v.viol_desc ASC
;'''

pd.read_sql(sql8, conn)

,action_desc,viol_desc
0,Establishment Closed by DOHMH. Violations wer...,''''No Smoking and/or 'Smoking Permitted sig...
1,Establishment re-closed by DOHMH,''''No Smoking and/or 'Smoking Permitted sig...
2,Establishment re-opened by DOHMH,''''No Smoking and/or 'Smoking Permitted sig...
3,Establishment re-closed by DOHMH,''''Wash hands sign not posted at hand wash f...
4,No violations were recorded at the time of thi...,''''Wash hands sign not posted at hand wash f...
...,...,...
216,Establishment re-closed by DOHMH,"Whole frozen poultry or poultry breasts, other..."
217,Establishment re-opened by DOHMH,"Whole frozen poultry or poultry breasts, other..."
218,No violations were recorded at the time of thi...,"Whole frozen poultry or poultry breasts, other..."
219,Establishment re-closed by DOHMH,Wiping cloths soiled or not stored in sanitizi...


In [23]:
my_sql['q8']['sql'] = sql8

---

9) For all restaurants (by camis) that have been cited for 90 or more violations, what is their top violation?

Have your query return:
- camis
- dba
- viol_id
- viol_desc
- Number of violations (for the top violation only)

Order the results in ascending order by dba


In [24]:
sql9 = '''
WITH MoreThan90 AS (
    SELECT camis, dba, COUNT(*) AS NumViol
    FROM tRest
    JOIN tInsp USING (camis)
    JOIN tViol USING (viol_id)
    GROUP BY camis
    HAVING NumViol >= 90
),
TopViolations AS (
    SELECT m.camis, m.dba, viol_id, viol_desc, COUNT(*) as ViolationCount,
           ROW_NUMBER() OVER (PARTITION BY m.camis ORDER BY COUNT(*) DESC) AS rn
    FROM MoreThan90 m
    JOIN tInsp USING(camis)
    JOIN tViol USING (viol_id)
    GROUP BY m.camis, viol_id, viol_desc
)
SELECT camis, dba, viol_id, viol_desc, ViolationCount
FROM TopViolations
WHERE rn = 1
ORDER BY dba ASC

;'''

pd.read_sql(sql9, conn)


,camis,dba,viol_id,viol_desc,ViolationCount
0,50001880,GANG SAN DEUL,02B,Hot food item not held at or above 140Âº F.,10
1,40965177,LA GATA GOLOSA #2,06C,Food not protected from potential source of co...,8
2,41683816,MAX BAKERY & RESTAURANT,04M,Live roaches present in facility's food and/or...,12
3,50033122,PARTY WELL REST & ORIENTAL BAKERY,08A,Facility not vermin proof. Harborage or condit...,14
4,41459659,T. K. KITCHEN,08A,Facility not vermin proof. Harborage or condit...,13


In [25]:
my_sql['q9']['sql'] = sql9

In [26]:
with open('midterm.data302', 'w') as f:
    json.dump(my_sql, f)

In [27]:
dbga.validate_submission_format(path_to_db,
                                path_submission = 'midterm.data302',
                                num_questions = 9)

/home/jovyan/Databases/databases/dbga.py:1119: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ).applymap(lambda y: y[:max_str_len] if isinstance(y, str) else y)
/home/jovyan/Databases/databases/dbga.py:1123: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ).applymap(lambda y: y[:max_str_len] if isinstance(y, str) else y)
/home/jovyan/Databases/databases/dbga.py:1119: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ).applymap(lambda y: y[:max_str_len] if isinstance(y, str) else y)
/home/jovyan/Databases/databases/dbga.py:1123: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ).applymap(lambda y: y[:max_str_len] if isinstance(y, str) else y)
/home/jovyan/Databases/databases/dbga.py:1132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ).applymap(lambda y: y[:max_str_len] if isinstance(y, str) else y).to_string(

Report saved to:  VALIDATION_midterm.txt


/home/jovyan/Databases/databases/dbga.py:1132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ).applymap(lambda y: y[:max_str_len] if isinstance(y, str) else y).to_string(index=False)


In [28]:
conn.close

<function Connection.close()>